# 要約 
このJupyter Notebookは、LMSYS - Chatbot Arenaコンペティションに関連する問題に取り組んでいます。具体的には、ユーザーが選択する応答の優劣を予測するための機械学習モデルを構築することを目指しています。このタスクは、異なるチャットボットからの応答のどちらが好まれるかを学習するものであり、ユーザーの好みを精度良く予測することが求められています。

### 使用されている手法とライブラリ
1. **ライブラリ**:
   - `transformers`: 事前学習済みのLLM（Llamaモデル）の読み込みと使用に利用。
   - `peft`: LoRA（Low-Rank Adaptation）技術を用いてモデルを効率的にトレーニングするためのライブラリ。
   - `datasets`: データセットの作成と扱いに使用。

2. **手法**:
   - **LoRA**: トレーニング時にメモリと計算資源を削減するための手法であり、モデルの特定の層にアダプタを追加してパラメータを微調整。
   - **SFTTrainer**: シーケンス分類タスクに特化したトレーナーを使用して、モデルへ入力データを供給し、訓練を行う。
   - **カスタムトークナイザー**: プロンプトと応答を適切な形式に整形し、モデルに供給する際にトークン化を行うクラスを実装。

### ワークフローの概要
1. 依存ライブラリのインストール。
2. モデルとデータの設定を行い、データの前処理（欠損値の削除、重複応答の削除等）を実施。
3. データセットからトレーニングと評価用のインデックスを生成し、Foldクロスバリデーションの準備。
4. LoRAの設定を適用し、モデルを準備してトレーニングを開始。
5. トレーニング後、各Foldに対して検証を行い、評価結果を出力。

このNotebookは、特に事前学習済みモデルの効率的な微調整を通じて、競合する応答の選好を予測するための強力なフレームワークを提供しています。

---


# 用語概説 
以下は、提示されたJupyter Notebookの内容に基づいて、機械学習・深層学習の初心者がつまずきそうな専門用語の簡単な解説です。初心者向けの一般的な説明は省略し、特にノートブック内で使用されている特有の用語や概念に焦点を当てています。

1. **bitsandbytes**: メモリの使用量を最適化するためのライブラリで、特に大規模なモデルを小さいメモリFootprintで運用できるように設計されています。通常、ハードウェアリソースの制約で扱えないサイズのモデルを扱うために使用されます。

2. **PEFT (Parameter-Efficient Fine-Tuning)**: 機械学習モデルのファインチューニングの一手法で、大規模なモデルの特定のパラメータのみを調整することで、学習リソースを効率的に使用することを目的としています。これにより、大規模モデルを小さいデータセットで迅速に適応させることが可能になります。

3. **Lora (Low-Rank Adaptation)**: PEFTの一形式で、モデルの重みを低ランクの行列で近似することにより、必要なパラメータの数を大幅に減らすアプローチです。これにより、モデルのトレーニングがより効率的に行えます。

4. **gradient accumulation**: 複数のバッチにわたって勾配を蓄積してからモデルを更新する手法です。これにより、大きなバッチサイズを個別のGPUメモリではなく、より少ないメモリ使用量で模倣することができます。

5. **torch_dtype**: PyTorchにおいて、テンソルのデータ型を指定するための引数です。`torch.float16`を指定することで、データの精度を落としながら計算速度を速くし、メモリ使用量を削減することができます。

6. **SFT (Supervised Fine-Tuning)**: 教師あり学習によるファインチューニングの方法で、訓練データを使用してモデルの性能を向上させることを目的としています。このプロセスは、特定のタスクに対してモデルのパフォーマンスを最適化するのに役立ちます。

7. **EvalPrediction**: 評価の際に使用されるデータ構造で、モデルの予測結果と対応するラベルを格納します。このオブジェクトは、評価メトリクスの計算やモデルの性能を測定するのに使用されます。

8. **Kビットトレーニング**: モデルの重みを低ビット精度（例: 4ビットなど）で保存・計算することにより、大規模なパラメータを持つモデルをメモリに収容できるようにするトレーニング手法です。メモリ帯域幅やストレージの節約に貢献します。

9. **Llama**: 特定の大規模言語モデルやアーキテクチャを指すもので、この文脈では特に「LlamaForSequenceClassification」や「LlamaTokenizerFast」として現れることが多いです。このモデルは、シーケンス分類タスクに特化しています。

10. **DataCollatorWithPadding**: バッチ処理の際にデータにパディングを追加し、各入力データが同じサイズになるように整形するためのオブジェクトです。シーケンスデータを扱う際に重要です。

これらの用語や概念は、実務経験の不足から初心者にとっては難解に感じられることがあるため、しっかり理解しておくことが重要です。

---


In [ ]:
!pip install -q -U bitsandbytes --no-index --find-links ../input/llama3-1-dependencies/dependencies/
!pip install -q -U transformers --no-index --find-links ../input/llama3-1-dependencies/dependencies/
!pip install -q -U tokenizers --no-index --find-links ../input/llama3-1-dependencies/dependencies/
!pip install -q -U peft --no-index --find-links ../input/llama3-1-dependencies/dependencies/

In [ ]:
!pip install -U trl

In [ ]:
# このPython 3環境には多くの役立つ分析ライブラリがインストールされています。
# この環境はkaggle/python Dockerイメージによって定義されています: https://github.com/kaggle/docker-python
# たとえば、以下は読み込むのに役立つパッケージのいくつかです。

import numpy as np # 線形 algebra（線形代数）
import pandas as pd # データ処理、CSVファイルの入出力（例: pd.read_csv）

# 入力データファイルは読み取り専用の"../input/"ディレクトリにあります。
# 例えば、これを実行すると（実行ボタンをクリックするかShift+Enterを押すことで）入力ディレクトリ内のすべてのファイルがリスト表示されます。

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# 現在のディレクトリ（/kaggle/working/）には最大20GBまで書き込むことができ、
# これは「すべて保存して実行」ボタンを使用してバージョンを作成すると出力として保持されます。
# また、一時ファイルは/kaggle/temp/に書き込むこともできますが、現在のセッションの外部には保存されません。

In [ ]:
import os
import copy
from dataclasses import dataclass

import numpy as np
import torch
from datasets import Dataset
from transformers import (
    DataCollatorWithPadding,
    LlamaForSequenceClassification,
    LlamaTokenizerFast,
    PreTrainedTokenizerBase,
    EvalPrediction,
)
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training, TaskType, PeftModel
from sklearn.metrics import log_loss, accuracy_score

In [ ]:
@dataclass
class Config:
    output_dir: str = "output"  # 出力ディレクトリ
    checkpoint: str = "/kaggle/input/unsloth-meta-llama-3.1-8b-bnb-4bit/transformers/default/1/Meta-Llama-3.1-8B-bnb-4bit"  # チェックポイントのパス
    max_length: int = 2048  # 最大入力長
    n_splits: int = 5  # データを分割する数
    fold_idx: int = 0  # フォールドのインデックス
    optim_type: str = "adamw_8bit"  # 最適化アルゴリズムの種類
    per_device_train_batch_size: int = 4  # デバイスごとのトレーニングバッチサイズ
    gradient_accumulation_steps: int = 4  # 勾配蓄積ステップ
    per_device_eval_batch_size: int = 8  # デバイスごとの評価バッチサイズ
    n_epochs: int = 1  # エポック数
    freeze_layers: int = 16  # 合計32層のうち、最初の16層はアダプタを追加しない
    lr: float = 2e-4  # 学習率
    warmup_steps: int = 20  # ウォームアップステップ数
    lora_r: int = 4  # LoRAの次元数
    lora_alpha: float = lora_r * 2  # LoRAのスケーリング係数
    lora_dropout: float = 0.05  # LoRAのドロップアウト率
    lora_bias: str = "none"  # LoRAのバイアスの種類

config = Config()  # 設定クラスのインスタンスを生成

In [ ]:
lora_config = LoraConfig(
    r=config.lora_r,  # LoRAの次元数
    lora_alpha=config.lora_alpha,  # LoRAのスケーリング係数
    target_modules=["q_proj", "k_proj", "v_proj"],  # 変更対象のモジュール
    layers_to_transform=[i for i in range(32) if i >= config.freeze_layers],  # 変換対象層
    lora_dropout=config.lora_dropout,  # LoRAのドロップアウト率
    bias=config.lora_bias,  # LoRAのバイアスの種類
    task_type=TaskType.SEQ_CLS,  # タスクの種類（シーケンス分類）
)

In [ ]:
tokenizer = LlamaTokenizerFast.from_pretrained(config.checkpoint)  # トークナイザーの初期化
tokenizer.pad_token_id = tokenizer.eos_token_id  # パディングトークンIDをEOSトークンIDに設定
tokenizer.pad_token = tokenizer.eos_token  # パディングトークンをEOSトークンに設定
tokenizer.add_eos_token = True  # テキストの最後に<EOS>を追加
tokenizer.padding_side = "right"  # パディングの追加位置を右側に設定

In [ ]:
model = LlamaForSequenceClassification.from_pretrained(
    config.checkpoint,
    num_labels=3,  # 分類するラベルの数
    torch_dtype=torch.float16,  # 使用するデータ型
    device_map="auto"  # デバイスマッピングを自動決定
)

In [ ]:
model.config.use_cache = False  # キャッシュの使用を無効にする
model = prepare_model_for_kbit_training(model)  # Kビットトレーニングのためにモデルを準備
model = get_peft_model(model, lora_config)  # PEFTモデルを取得
model  # モデルの状態を確認

In [ ]:
model.print_trainable_parameters()  # トレーニング可能なパラメータを表示

In [ ]:
model.config.pad_token_id = tokenizer.pad_token_id  # パディングトークンIDを設定
model.config.use_cache = False  # キャッシュの使用を無効にする
model.config.pretraining_tp = 1  # プリトレーニングのTPを1に設定

In [ ]:
import pandas as pd
df = pd.read_csv('/kaggle/input/lmsys-chatbot-arena/train.csv')  # CSVファイルを読み込む
df = df.dropna()  # 欠損値を削除
df = df.drop_duplicates(subset=['response_a', 'response_b'], keep=False)  # 重複した応答を削除
df["len"] = df["prompt"].apply(len) + df["response_a"].apply(len) + df["response_b"].apply(len)  # 総長を計算
df = df.sort_values(by=['len'])  # 長さでソート
df  # データフレームを表示

In [ ]:
ds = Dataset.from_pandas(df)  # pandas DataFrameからデータセットを作成
ds = ds.select(torch.arange(1000)) # デモ目的のために1000行を選択

In [ ]:
class CustomTokenizer:
    def __init__(
        self, 
        tokenizer: PreTrainedTokenizerBase,  # トークナイザー
        max_length: int  # 最大長
    ) -> None:
        self.tokenizer = tokenizer  # トークナイザーを初期化
        self.max_length = max_length  # 最大長を設定
        
    def __call__(self, batch: dict) -> dict:  # バッチを處理するメソッド
        prompt = ["Which is the better response for the prompt? response_a or response_b or tie? \n'n give score for each lable \n\n <prompt>: " + self.process_text(t) for t in batch["prompt"]]  # プロンプトを構築
        response_a = ["\n\n<response_a>: " + self.process_text(t) for t in batch["response_a"]]  # 応答Aを処理
        response_b = ["\n\n<response_b>: " + self.process_text(t) for t in batch["response_b"]]  # 応答Bを処理
        texts = [p + r_a + r_b for p, r_a, r_b in zip(prompt, response_a, response_b)]  # テキストを結合
        tokenized = self.tokenizer(texts, max_length=self.max_length, truncation=True)  # トークン化
        labels=[]  # ラベルのリストを初期化
        for a_win, b_win in zip(batch["winner_model_a"], batch["winner_model_b"]):  # 各応答の勝者を評価
            if a_win:
                label = 0  # 応答Aが勝者
            elif b_win:
                label = 1  # 応答Bが勝者
            else:
                label = 2  # 引き分け
            labels.append(label)  # ラベルを追加
        return {**tokenized, "labels": labels}  # トークン化結果とラベルを返す
        
    @staticmethod
    def process_text(text: str) -> str:  # テキストを処理する静的メソッド
        return " ".join(eval(text, {"null": ""}))  # テキストを評価して文字列を結合

In [ ]:
encode = CustomTokenizer(tokenizer, max_length=config.max_length)  # カスタムトークナイザーのインスタンスを生成
ds = ds.map(encode, batched=True)  # データセットにトークナイザーを適用

In [ ]:
def compute_metrics(eval_preds: EvalPrediction) -> dict:  # 評価予測に基づいてメトリクスを計算する関数
    preds = eval_preds.predictions  # 予測結果を取得
    labels = eval_preds.label_ids  # ラベルを取得
    probs = torch.from_numpy(preds).float().softmax(-1).numpy()  # 確率を計算
    # 予測結果やラベルにNaNが含まれていないか確認
    if np.isnan(probs).any() or np.isnan(labels).any():
        raise ValueError("NaN values found in predictions or labels")  # NaNが見つかった場合はエラーをスロー

    loss = log_loss(y_true=labels, y_pred=probs)  # ログ損失を計算
    acc = accuracy_score(y_true=labels, y_pred=preds.argmax(-1))  # 正確度を計算
    return {"acc": acc, "log_loss": loss}  # メトリクスを返す

In [ ]:
folds = [
        (
            [i for i in range(len(ds)) if i % config.n_splits != fold_idx],  # トレーニング用インデックス
            [i for i in range(len(ds)) if i % config.n_splits == fold_idx]  # 評価用インデックス
        ) 
        for fold_idx in range(config.n_splits)  # フォールドインデックスに基づいて分割
    ]

In [ ]:
from trl import SFTTrainer, SFTConfig  # SFTHandlerと設定をインポートする
sft_config = SFTConfig(
    output_dir="output",  # 出力ディレクトリ
    overwrite_output_dir=True,  # 出力ディレクトリを上書き
    report_to="none",  # レポートを行わない
    num_train_epochs=config.n_epochs,  # 学習エポック数
    per_device_train_batch_size=config.per_device_train_batch_size,  # デバイスごとのトレーニングバッチサイズ
    gradient_accumulation_steps=config.gradient_accumulation_steps,  # 勾配蓄積のステップ数
    per_device_eval_batch_size=config.per_device_eval_batch_size,  # デバイスごとの評価バッチサイズ
    logging_steps=1000,  # ロギングのステップ数
    save_strategy="epoch",  # エポックごとにモデルを保存
    save_steps=100,  # 100ステップごとに保存
    optim=config.optim_type,  # 最適化アルゴリズムの種類
    fp16=True,  # 半精度浮動小数点を使用する
    learning_rate=config.lr,  # 学習率
    warmup_steps=config.warmup_steps,  # ウォームアップステップ数
    packing=True,  # Packingを有効にする
    dataset_text_field="text",  # データセットのテキストフィールド名
    max_seq_length=config.max_length,  # 最大シーケンス長
)

In [ ]:
trainer = SFTTrainer(
        model,
        train_dataset=ds,  # トレーニングデータセット
        args=sft_config  # トレーニング設定
    )

In [ ]:
for fold_idx in range(config.n_splits):  # 各フォールドに対して
    train_idx, eval_idx = folds[fold_idx]  # トレーニングと評価のインデックスを取得

    train_data = ds.select(train_idx).sort("len")  # トレーニングデータを選択してソート
    val_data = ds.select(eval_idx).sort("len")  # 評価データを選択してソート
    
    # 同じ長さの範囲でトレーニングデータをバッチに分割
    batch_size = 200  # バッチサイズ
    num_batches = len(train_data) // batch_size + (1 if len(train_data) % batch_size != 0 else 0)  # バッチ数を計算
    
    for batch_idx in range(num_batches):  # 各バッチに対して
        start_idx = batch_idx * batch_size  # 開始インデックス
        end_idx = min(start_idx + batch_size, len(train_data))  # 終了インデックス
        ds_temp = train_data.select(range(start_idx, end_idx))  # 一時的なデータセットを作成
        
        trainer.train_dataset = ds_temp  # トレーナーに一時データセットを設定
        
        print(f"Training batch {batch_idx + 1}/{num_batches} on fold {fold_idx + 1}/{config.n_splits}...")  # トレーニング進行状況を表示
        
        trainer.train()  # モデルをトレーニング
        
        trainer.save_model(f"model_fold_{fold_idx}_batch{batch_idx}")  # モデルを保存

    
    # すべてのバッチでのトレーニング後に検証
    trainer.eval_dataset = val_data  # 評価データセットを設定
    
    print(f"Validating on fold {fold_idx + 1}/{config.n_splits}...")  # 検証進行状況を表示
    eval_results = trainer.evaluate()  # モデルを評価します

    # メトリクスが必要な場合に保存
    print(f"Evaluation results for fold {fold_idx + 1}: {eval_results}")  # 評価結果を表示

---

# コメント 

> ## PaulRRR
> 
> こんにちは、推論コードはありますか？
> 
> 
> 


---